In [10]:
#@title Install required packages
!uv pip uninstall "." # proxy structuring engine
!uv pip install -U ".[dev]" # proxy structuring engine
!uv pip install sentencepiece
!uv pip install accelerate
!uv pip install transformers
!uv pip install bitsandbytes
!uv pip install sentencepiece
!uv pip install protobuf
!uv pip install -U tqdm
!uv pip install ipywidgets

Uninstalled 1 package in 4ms
 - pse==1.5.9 (from file:///Users/jckwind/Documents/proxy-structuring-engine)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


⠙ pse==1.5.9                                                                    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 73 packages in 268ms                                        
Building pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
   Built pse @ file:///Users/jckwind/Documents/proxy-structuring-engine
Prepared 1 package in 178ms                                              
Installed 1 package in 1msle:///Users/jckwind/Documents/prox
 + pse==1.5.9 (from file:///Users/jckwind/Documents/proxy-structuring-engine)
Audited 1 package in 1ms


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 1 package in 35ms                                           
Audited 1 package in 0.08ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Audited 1 package in 1ms


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
#@title Setup Llama 3.2 1B and create the structuring engine
import torch
from transformers import (
    AutoTokenizer,
    LlamaForCausalLM,
    LogitsProcessorList,
)

from pse.engine.structuring_engine import StructuringEngine

model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
engine = StructuringEngine(tokenizer)
logits_processor = LogitsProcessorList([
    engine.process_logits
])

model = LlamaForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.pad_token_id = model.config.eos_token_id[0]
if model.generation_config:
    model.generation_config.pad_token_id = model.config.pad_token_id or model.config.eos_token_id[0]
    model.generation_config.max_new_tokens = 1000

In [12]:

#@title Test json generation
import json

SIMPLE_JSON_SCHEMA = {
    "type": "object",
    "properties": {"value": {"type": "number"}},
    "required": ["value"],
}
engine.configure(SIMPLE_JSON_SCHEMA)
prompt = f"Please generate the number 9.11 in a json object with the following schema:\n{json.dumps(SIMPLE_JSON_SCHEMA, indent=2)}"
messages = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True)
assert isinstance(input_ids, torch.Tensor)
input_ids = input_ids.to(model.device)
assert isinstance(input_ids, torch.Tensor)
greedy_output = model.generate(
    input_ids,
    logits_processor=logits_processor,
    do_sample=True,
)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0]))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!